Turn on GPU. 

Number of connected components of the NN (nearest neigbour) graph for uniformly sampled points in [0,1]^d (newer version simulate for Gassian  ) is estimated by simulation.
(It grows linearly with point number, so we are interested in that coefficient of linear dependence).
Theoretical esimtations are discussed at:
https://cstheory.stackexchange.com/questions/47034/number-of-connected-components-of-a-random-nearest-neighbor-graph

Our simulations supports claim proposed by Neal Young for sampling in sphere or torus, but seems contradict that claim for uniformly or gaussianly distributed points. 

We see some desrepancy (for high d) between theoretical estimates proposed  and numerical simulations here.
It might be that we are not simulating large enough number of points.
For small d, like d=2 simulation and theory fit each other. (The result for d=2 has been published in 2007, while higher d proposed just in the discussion above, has not yet been published ). 

We use GPU to accelerate the calculations - "RAPIDS package"

PS

By connected component we mean "weakly" connected component - in the other words  forgeting orientation



V20 - torus 

V19 - sphere - repeat 

V18 - correction of V17 - to generate d-sphere we should take points in d+1 space - that affects only small dimensions, but still better to recalculate  

V17 - new simulation for sphere (random points on sphere) 

V16 - new simulation for uniform data cloud for several dimensions, sizes of data clouds and each is simulated 100 times

V14,15 - new simulation for Gaussian data cloud  for several dimensions, sizes of data clouds and each is simulated 100 times. (V15 was planned to be for uniform, but due to misprint appeared to be Gaussian again). 


V11,12,13 - test save 

Version 10  - Gaussian Dimension 10 

In [ ]:
# Use rapids - https://www.kaggle.com/cdeotte/rapids - some GPU acceleration for many algorithms
# 
# Current notebook  is based on Dmitry Simakov notebook: https://www.kaggle.com/simakov/rapids-knn-cugraph-test

if 0:  # That was necessary to do in the past - now we can just import cudf, cuml 
    # That cell may run 3-5 minutes. (And sometimes may hang on - if so - restart notebook and run again )
    import sys
    !cp ../input/rapids/rapids.0.13.0 /opt/conda/envs/rapids.tar.gz
    !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
    sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
    sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
    sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
    !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/


try:
    import cudf
except:
    print('Turn ON GPU ! Otherwise will not work')
#import cugraph
from cuml.neighbors import NearestNeighbors as cuNearestNeighbors
import numpy as np
import igraph
import time
import pandas as pd


In [ ]:
data_type = 'torus' # 'sphere' # 'uniform'# 'Gaussian'
n_trials = 100
list_sample_sizes =  [1e3,1e4,1e5, 1e6]
list_dim = [2,5, 10,15, 20, 50, 100]
verbose = 10



t00 = time.time()
t0 = time.time()
df_stat = pd.DataFrame()
c = 0

timer_for_print = time.time()
time_interval_for_print = 10 # seconds

for dim in list_dim:
    if data_type == 'sphere':
        dim += 1 # n-Sphere is constructed from R^{n+1}  
        
    for n_sample in list_sample_sizes: # [1e3,1e4]:# ,1e5,1e6,1e7]:
        n_sample = int(n_sample)


        n_neighbors = 2
        for i in range(n_trials):
            t0 = time.time()
            c+=1
            df_stat.loc[c,'Dim'] = dim
            df_stat.loc[c,'sample size'] = n_sample

            np.random.seed(n_sample + i)
            if data_type == 'uniform':
                X = np.random.rand(n_sample, dim) # uniform
            elif data_type == 'sphere':
                X = np.random.randn(n_sample, dim) # Gaussian
                nm = np.sqrt( np.sum(X*X,axis = 1)  )
                X = X / nm[:,np.newaxis]
            elif data_type == 'torus':
                X = ( np.random.rand(n_sample,dim) )
                X1 = np.sin(2*np.pi*X)
                X2 = np.cos(2*np.pi*X)
                X = np.concatenate((X1,X2),axis=1)                
            else:
                X = np.random.randn(n_sample, dim) # Gaussian
                
            device_data = cudf.DataFrame(X) # cudf.DataFrame.from_gpu_matrix(X)

            knn_cuml = cuNearestNeighbors(n_neighbors)
            knn_cuml.fit(device_data)
            D_cuml, I_cuml = knn_cuml.kneighbors(device_data, n_neighbors)


            g = igraph.Graph(directed = True)
            g.add_vertices(range(n_sample))
            g.add_edges(I_cuml.to_pandas().values)
            r = g.clusters(mode = 'WEAK')

            df_stat.loc[c,'%Coeff'] = len(r) / n_sample * 100

            comp_size = [len(tmp) for tmp in r ]
            df_stat.loc[c,'Max comp size'] = np.max(comp_size)
            df_stat.loc[c,'Mean comp size'] = np.mean(comp_size)
            df_stat.loc[c,'Median comp size'] = np.median(comp_size)
            df_stat.loc[c,'Seconds passed'] = time.time() - t0
            df_stat.loc[c,'n_trials'] = n_trials
            #print(np.max(comp_size))
            if (verbose >= 1) and ( timer_for_print - time.time() >= time_interval_for_print ) :
                time_interval_for_print = time.time()
                print('c',c,'i',i,'dim', dim, 'n_sample', n_sample , np.round(time.time()-t00,1), 'seconds passed' )
            #print(df_stat.tail(1))
        if verbose >= 10:
            print('Finsihed  for n_sample', n_sample, 'c',c,'i',i,'dim', dim, 'n_sample', n_sample , np.round(time.time()-t00,1), 'seconds passed' )
    if verbose >= 1:
        print('Finsihed  for dimension', dim,'c',c,'i',i,'dim', dim, 'n_sample', n_sample , np.round(time.time()-t00,1), 'seconds passed' )
    

print(np.round(time.time()-t00,1), np.round( (time.time()-t00)/60,1),np.round( (time.time()-t00)/3600,1),'seconds, minutes, hours passed')
df_stat.groupby(['Dim', 'sample size']).aggregate(['mean','std']) 
#df_stat

# Main result

The formula for the number of connected components divided by number of nodes  for random NN graph
proposed by  Neal Young: 
https://cstheory.stackexchange.com/questions/47034/number-of-connected-components-of-a-random-nearest-neighbor-graph
(The answer depends on dimension.)

Empiric simulation for it (we show "percent" i.e. we multiplied by 100)  is given by "%Coef" in the table below.

Neal Young formula predicts that "%Coef" tends to 0.25 for high enough dimension.

Our simulations supports that claim for sampling in sphere or torus, but seems contradict that claim for uniformly or gaussianly distributed points. 


In [ ]:
print(data_type)

df_stat.groupby(['Dim', 'sample size'])[['%Coeff','n_trials']].aggregate(['mean','std']) 


# More detailed info



In [ ]:
print(data_type)
df_aggregated_stat = df_stat.groupby(['Dim', 'sample size']).aggregate(['mean','std']) 
df_aggregated_stat.to_csv('df_aggregated_stat.csv')
df_aggregated_stat

In [ ]:
df_stat.groupby(['Dim', 'sample size']).mean()# aggregate(['mean','std']) 

In [ ]:
df_stat.groupby(['Dim', 'sample size']).std()# aggregate(['mean','std']) 

In [ ]:
df_stat.to_csv('df_stat.csv')
df_stat